In [1]:
#Load data into sqlite database
import numpy as np
import pandas as pd
import sqlite3

In [2]:
pd.set_option('display.max_columns', None)

In [4]:
db = sqlite3.connect('./data/npidata_final.sqlite')

query = "SELECT * FROM nppes LIMIT 5"

hopteaming = pd.read_sql(query, db)

db.close()

In [3]:
hopteaming


,COUNT(Entity_Type_Code)
0,17493628


In [22]:
hopteaming.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 48 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   NPI                                                      5 non-null      int64  
 1   Entity_Type_Code                                         4 non-null      float64
 2   Provider Organization Name (Legal Business Name)         1 non-null      object 
 3   Provider Last Name (Legal Name)                          3 non-null      object 
 4   Provider First Name                                      3 non-null      object 
 5   Provider Middle Name                                     2 non-null      object 
 6   Provider Name Prefix Text                                2 non-null      object 
 7   Provider Name Suffix Text                                0 non-null      object 
 8   Provider Credential Text          

In [21]:
db = sqlite3.connect('./data/npidata_final.sqlite')

query = "SELECT * FROM npi LIMIT 5"

npi = pd.read_sql(query, db)

db.close()

In [5]:
npi.head()

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1508085911,1730166125,58,67,23.925,43.923
1,1508167040,1730166125,51,51,28.196,52.876
2,1508863549,1730166125,340,391,18.302,42.422
3,1508867870,1730166125,50,79,12.658,26.402
4,1508011040,1730166224,132,145,8.579,28.053


In [6]:
npi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   from_npi           5 non-null      int64  
 1   to_npi             5 non-null      int64  
 2   patient_count      5 non-null      int64  
 3   transaction_count  5 non-null      int64  
 4   average_day_wait   5 non-null      float64
 5   std_day_wait       5 non-null      float64
dtypes: float64(2), int64(4)
memory usage: 372.0 bytes


In [12]:
db = sqlite3.connect('./data/npidata_final.sqlite')

query = 'SELECT "Provider First Line Business Practice Location Address" FROM nppes'

rename_test = pd.read_sql(query, db)

db.close()

In [29]:
rename_test.head()

,Provider First Line Business Practice Location Address
0,3500 CENTRAL AVE
1,1824 KING STREET
2,3418 VILLAGE DR
3,None
4,17323 RED OAK DR


First, build a profile of providers referring patients to the major hospitals in Nashville. 

Are certain specialties more likely to refer to a particular hospital over the others?

In [ ]:
#First what are the major Nashville Hospitals?
    #Vanderbilt
    #Centennial
    #St. Thomas (Midtown, West)
    #Nashville General
    #VA
#Next what are their major providers?
#What are the specialties we would be interested in here. 
   #How focused do we need to be on 'hospitals'
    #We could select out the hospitals
    #

In [ ]:
#First select out all practices in Nashville
#Then select out important information from npi's
#Then join them together
#Next, what are the major Nashville Hospitals?
    #Vanderbilt
    #Centennial
    #St. Thomas (midtown, west)
    #Nashville General
    #VA

In [25]:
db = sqlite3.connect('./data/npidata_final.sqlite')

query = 'SELECT "from_npi", "to_npi", "patient_count" FROM npi'

npis_patient_count = pd.read_sql(query, db)

db.close()

In [26]:
npis_patient_count.head(2)


,from_npi,to_npi,patient_count
0,1508085911,1730166125,58
1,1508167040,1730166125,51


In [51]:
#Merge df's to find the largest to_npis by patient count
#Using to_npi as the merge key, and look through the resulting df, to find largest patient counts.  
largest_by_patient_count = pd.merge(Nashville_Practices, npis_patient_count, how='left', left_on='NPI', right_on='to_npi')

In [52]:
largest_by_patient_count.head(2)

,Provider Organization Name (Legal Business Name),Provider Business Practice Location Address City Name,NPI,Classification,Taxonomy_Code,from_npi,to_npi,patient_count
0,None,NASHVILLE,1134122187,Nurse Practitioner,363L00000X,1.780688e+09,1.134122e+09,52.0
1,None,NASHVILLE,1134122187,Nurse Practitioner,363L00000X,1.568465e+09,1.134122e+09,52.0


In [53]:
summed_patient_count = largest_by_patient_count.groupby(['NPI'])['patient_count'].sum().reset_index()

In [54]:
summed_patient_count.sort_values('patient_count', ascending=False)

,NPI,patient_count
208,1003863580,3462260.0
2926,1124075635,2345798.0
2495,1104202761,2329254.0
5619,1235186800,2145924.0
10592,1437194669,2049324.0
...,...,...
9090,1376203703,0.0
9089,1376196717,0.0
9088,1376181636,0.0
9087,1376177972,0.0


In [ ]:
#Use the CBSA code 34980 for finding Nashville Area providers

In [5]:
hop_teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33563184 entries, 0 to 33563183
Data columns (total 6 columns):
 #   Column             Dtype  
---  ------             -----  
 0   from_npi           int64  
 1   to_npi             int64  
 2   patient_count      int64  
 3   transaction_count  int64  
 4   average_day_wait   float64
 5   std_day_wait       float64
dtypes: float64(2), int64(4)
memory usage: 1.5 GB


In [6]:
db = sqlite3.connect('./data/npidata_final.sqlite')

query = '''SELECT * 
FROM nppes 
WHERE "Provider Organization Name (Legal Business Name)" 
LIKE "%Vanderbilt%" 
AND "Classification" 
LIKE "%hospital%" 
AND "CBSA" ="34980" 
GROUP BY "Provider Organization Name (Legal Business Name)" '''

Vanderbilt_Hospitals = pd.read_sql(query, db)

db.close()

In [ ]:
# You could select out from NPPES hospitals by what? 

In [7]:
Vanderbilt_Hospitals

,NPI,Entity_Type_Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Healthcare Provider Taxonomy Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,Taxonomy_Code,Classification,zipcodes,CBSA
0,1255577466,2.0,MONROE CARELL JR VANDERBILT CHILDREN'S HOSPITAL,None,None,None,None,None,None,2200 CHILDRENS WAY,SUITE 4150,NASHVILLE,TN,372320005.0,282NC2000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282NC2000X,General Acute Care Hospital,37232,34980
1,1003211145,2.0,MONROE CARELL JR. CHILDREN'S HOSPITAL AT VANDE...,None,None,None,None,None,None,2200 CHILDRENS WAY PRESTON RESEARCH BLDG 395,None,NASHVILLE,TN,372320005.0,282NC2000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282NC2000X,General Acute Care Hospital,37232,34980
2,1487253449,2.0,"VANDERBILT BEDFORD HOSPITAL, LLC",None,None,None,None,None,None,2835 HIGHWAY 231 N,None,SHELBYVILLE,TN,371607327.0,282N00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282N00000X,General Acute Care Hospital,37160,34980
3,1609056399,2.0,VANDERBILT CHILDREN'S,None,None,None,None,None,None,1702 THE VANDERBILT CLINIC,1301 22ND AVE SOUTH,NASHVILLE,TN,372320001.0,282NC2000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282NC2000X,General Acute Care Hospital,37232,34980
4,1326224015,2.0,VANDERBILT CHILDRENS HOSPITAL,None,None,None,None,None,None,2200 CHILDRENS WAY STE 3115,None,NASHVILLE,TN,372320005.0,282NC2000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282NC2000X,General Acute Care Hospital,37232,34980
5,1619040813,2.0,VANDERBILT COMMUNITY MENTAL HEALTH CENTER,None,None,None,None,None,None,1601 23RD AVE S,3RD FLOOR,NASHVILLE,TN,372123133.0,283Q00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,283Q00000X,Psychiatric Hospital,37212,34980
6,1992770119,2.0,VANDERBILT MEDICAL CENTER,None,None,None,None,None,None,S-3414 MEDICAL CENTER NORTH,C/O INTERNAL MEDICINE DEPT,NA

In [8]:
db = sqlite3.connect('./data/npidata_final.sqlite')

query = ''' SELECT * 
FROM nppes 
WHERE "Provider Organization Name (Legal Business Name)" LIKE "%Centennial%" 
AND "Classification" like "%hospital%" 
AND "CBSA" ="34980" 
GROUP BY "Provider Organization Name (Legal Business Name)" ''' 

Centennial_Hospitals = pd.read_sql(query, db)

db.close()

In [9]:
Centennial_Hospitals

,NPI,Entity_Type_Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Healthcare Provider Taxonomy Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,Taxonomy_Code,Classification,zipcodes,CBSA
0,1710314554,2.0,CENTENNIAL,None,None,None,None,None,None,552 METROPLEX DR,None,NASHVILLE,TN,372113133.0,282N00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282N00000X,General Acute Care Hospital,37211,34980


In [10]:
db = sqlite3.connect('./data/npidata_final.sqlite')

query = ''' SELECT * 
FROM nppes 
WHERE "Provider Organization Name (Legal Business Name)" LIKE "%Saint Thomas%" 
AND "Classification" like "%hospital%" 
AND "CBSA" ="34980" GROUP BY "Provider Organization Name (Legal Business Name)" ''' 

Stthomas_Hospitals = pd.read_sql(query, db)

db.close()

In [11]:
Stthomas_Hospitals

,NPI,Entity_Type_Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Healthcare Provider Taxonomy Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,Taxonomy_Code,Classification,zipcodes,CBSA
0,1649659582,2.0,"SAINT THOMAS DEKALB HOSPITAL, LLC",None,None,None,None,None,None,520 W MAIN ST,None,SMITHVILLE,TN,371661138.0,282N00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282N00000X,General Acute Care Hospital,37166,34980
1,1871859348,2.0,SAINT THOMAS HEART,None,None,None,None,None,None,4220 HARDING PIKE,None,NASHVILLE,TN,372052005.0,282N00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282N00000X,General Acute Care Hospital,37205,34980
2,1699436626,2.0,"SAINT THOMAS REHABILITATION HOSPITAL, LLC",None,None,None,None,None,None,310 21ST AVE N,None,NASHVILLE,TN,372031846.0,283X00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,283X00000X,Rehabilitation Hospital,37203,34980
3,1518346469,2.0,"SAINT THOMAS RIVER PARK HOSPITAL, LLC",None,None,None,None,None,None,1559 SPARTA ST,None,MCMINNVILLE,TN,371101316.0,282N00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282N00000X,General Acute Care Hospital,37110,34980
4,1164590386,2.0,SAINT THOMAS RUTHERFORD HOSPITAL,None,None,None,None,None,None,1700 MEDICAL CENTER PKWY,None,MURFREESBORO,TN,371292245.0,282N00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282N00000X,General Acute Care Hospital,37129,34980
5,1447639398,2.0,"SAINT THOMAS STONES RIVER HOSPITAL, LLC",None,None,None,None,None,None,324 DOOLITTLE RD,None,WOODBURY,TN,371901139.0,273R00000X,N,282N00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282N00000X,General Acute Care Hospital,37190,34980
6,1629025648,2.0,SAINT THOMAS WEST HOSPITAL,None,None,None,None,None,None,4220 HARDING RD,None,NASHVILLE,TN,372052005.0,282N00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,No

In [12]:
db = sqlite3.connect('./data/npidata_final.sqlite')

query = ''' SELECT * 
FROM nppes 
WHERE "Provider Organization Name (Legal Business Name)" LIKE "%General%" 
AND "Classification" like "%hospital%" 
AND "CBSA" ="34980" 
GROUP BY "Provider Organization Name (Legal Business Name)" ''' 

Nashville_General = pd.read_sql(query, db)

db.close()

In [13]:
Nashville_General

,NPI,Entity_Type_Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Healthcare Provider Taxonomy Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,Taxonomy_Code,Classification,zipcodes,CBSA
0,1417938846,2.0,"MACON COUNTY GENERAL HOSPITAL, INC.",None,None,None,None,None,None,305 W LOCUST ST,None,LAFAYETTE,TN,370831712.0,282NC0060X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282NC0060X,General Acute Care Hospital,37083,34980
1,1568551455,2.0,NASHVILLE GENERAL HOSPITAL,None,None,None,None,None,None,1818 ALBION ST,ATTN: MARTHA LAMPLEY,NASHVILLE,TN,372082918.0,282N00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282N00000X,General Acute Care Hospital,37208,34980


In [14]:
db = sqlite3.connect('./data/npidata_final.sqlite')

query = ''' SELECT * 
FROM nppes 
WHERE "Provider Organization Name (Legal Business Name)" LIKE "%Veteran%" 
AND "Classification" like "%hospital%" 
AND "CBSA" ="34980" 
GROUP BY "Provider Organization Name (Legal Business Name)" ''' 

VA = pd.read_sql(query, db)

db.close()

In [15]:
VA

,NPI,Entity_Type_Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Healthcare Provider Taxonomy Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,Taxonomy_Code,Classification,zipcodes,CBSA
0,1568477990,2.0,DEPARTMENT OF VETERANS AFFAIRS NASHVILLE,None,None,None,None,None,None,1310 24TH AVE S,None,NASHVILLE,TN,372122637.0,286500000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,286500000X,Military Hospital,37212,34980
1,1609241389,2.0,VETERANS ADMINISTRATION,None,None,None,None,None,None,1015 W MAIN ST,None,FRANKLIN,TN,370642731.0,282N00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282N00000X,General Acute Care Hospital,37064,34980


In [32]:
#Once we decide on how specific to be (i.e. do we include only 'hospitals' or do we more add the network?)
#Then collect the NPI's and search for them using a 'WHERE' statement. 
#This will give us a good profile of providers referring to hospitals.  


In [ ]:
#Then, with the above profiles, look through the 'classification' column and see which specialities
#are the most likely to refer to hospitals.  

In [23]:
db = sqlite3.connect('./data/npidata_final.sqlite')

query = ''' SELECT * 
FROM nppes 
WHERE "classification" LIKE "%hospital%" 
AND "classification" NOT LIKE "Hospitalist" 
AND "Entity_Type_Code" NOT LIKE "1.0" AND "CBSA" ="34980" 
GROUP BY "Provider Organization Name (Legal Business Name)" ''' 

all_hospitals = pd.read_sql(query, db)

db.close()

### Thought:
##### Concentrate on the 5 major hospitals, but look at all the hospitals and use that as a way to compare who is referring and what might be a good area to look forward.  

##### Do the major hospitals look differently than the other hospitals, in specialties referring or in some type of distributions?

In [24]:
Vanderbilt_Hospitals.head(1)

,NPI,Entity_Type_Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Healthcare Provider Taxonomy Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,Taxonomy_Code,Classification,zipcodes,CBSA
0,1255577466,2.0,MONROE CARELL JR VANDERBILT CHILDREN'S HOSPITAL,None,None,None,None,None,None,2200 CHILDRENS WAY,SUITE 4150,NASHVILLE,TN,372320005.0,282NC2000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282NC2000X,General Acute Care Hospital,37232,34980


In [27]:
npis_patient_count.head(2)

,from_npi,to_npi,patient_count
0,1508085911,1730166125,58
1,1508167040,1730166125,51


In [29]:
vandy_merged=pd.merge(Vanderbilt_Hospitals, npis_patient_count, how='left', left_on='NPI', right_on='to_npi')

In [32]:
vandy_merged.head(1)

,NPI,Entity_Type_Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Healthcare Provider Taxonomy Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,Taxonomy_Code,Classification,zipcodes,CBSA,from_npi,to_npi,patient_count
0,1255577466,2.0,MONROE CARELL JR VANDERBILT CHILDREN'S HOSPITAL,None,None,None,None,None,None,2200 CHILDRENS WAY,SUITE 4150,NASHVILLE,TN,372320005.0,282NC2000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282NC2000X,General Acute Care Hospital,37232,34980,NaN,NaN,NaN


In [33]:
vandy_merged.groupby(['Provider Organization Name (Legal Business Name)'])['patient_count'].sum().reset_index()

,Provider Organization Name (Legal Business Name),patient_count
0,MONROE CARELL JR VANDERBILT CHILDREN'S HOSPITAL,0.0
1,MONROE CARELL JR. CHILDREN'S HOSPITAL AT VANDE...,0.0
2,"VANDERBILT BEDFORD HOSPITAL, LLC",0.0
3,VANDERBILT CHILDREN'S,0.0
4,VANDERBILT CHILDRENS HOSPITAL,0.0
5,VANDERBILT COMMUNITY MENTAL HEALTH CENTER,0.0
6,VANDERBILT MEDICAL CENTER,0.0
7,VANDERBILT ORTHOPAEDIC INSTITUTE,0.0
8,VANDERBILT PSYCHIATRIC HOSPITAL,0.0
9,"VANDERBILT PSYCHIATRIC HOSPITAL, LLC",0.0


In [39]:
hospitals_merged=pd.merge(hospitals, npis_patient_count, how='left', left_on='NPI', right_on='to_npi')

In [42]:
hospitals_merged.groupby(['Provider Organization Name (Legal Business Name)'])['patient_count'].sum().reset_index().sort_values('patient_count', ascending=False).head(20)


,Provider Organization Name (Legal Business Name),patient_count
24,MAURY REGIONAL HOSPITAL,327103.0
50,SAINT THOMAS WEST HOSPITAL,268557.0
48,SAINT THOMAS RUTHERFORD HOSPITAL,165738.0
90,WILLIAMSON COUNTY HOSPITAL DISTRICT,151878.0
63,SUMNER REGIONAL MEDICAL CENTER LLC,122222.0
21,HENDERSONVILLE HOSPITAL CORPORATION,104917.0
10,CENTRAL TENNESSEE HOSPITAL CORPORATION,79606.0
87,VANDERBILT UNIVERSITY MEDICAL CENTER,69346.0
19,"HCA HEALTH SERVICES OF TENNESSEE, INC.",66368.0
47,"SAINT THOMAS RIVER PARK HOSPITAL, LLC",59836.0


In [ ]:
#So most of these hospitals are NOT in Nashville.  However they provide a good overall inclusion to the provider network.  
#This could lead us to a good secondary network of providers to encourage referrals to. 


In [47]:
vandy_merged

,NPI,Entity_Type_Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Healthcare Provider Taxonomy Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,Taxonomy_Code,Classification,zipcodes,CBSA,from_npi,to_npi,patient_count
0,1255577466,2.0,MONROE CARELL JR VANDERBILT CHILDREN'S HOSPITAL,None,None,None,None,None,None,2200 CHILDRENS WAY,SUITE 4150,NASHVILLE,TN,372320005.0,282NC2000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282NC2000X,General Acute Care Hospital,37232,34980,NaN,NaN,NaN
1,1003211145,2.0,MONROE CARELL JR. CHILDREN'S HOSPITAL AT VANDE...,None,None,None,None,None,None,2200 CHILDRENS WAY PRESTON RESEARCH BLDG 395,None,NASHVILLE,TN,372320005.0,282NC2000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282NC2000X,General Acute Care Hospital,37232,34980,NaN,NaN,NaN
2,1487253449,2.0,"VANDERBILT BEDFORD HOSPITAL, LLC",None,None,None,None,None,None,2835 HIGHWAY 231 N,None,SHELBYVILLE,TN,371607327.0,282N00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282N00000X,General Acute Care Hospital,37160,34980,NaN,NaN,NaN
3,1609056399,2.0,VANDERBILT CHILDREN'S,None,None,None,None,None,None,1702 THE VANDERBILT CLINIC,1301 22ND AVE SOUTH,NASHVILLE,TN,372320001.0,282NC2000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282NC2000X,General Acute Care Hospital,37232,34980,NaN,NaN,NaN
4,1326224015,2.0,VANDERBILT CHILDRENS HOSPITAL,None,None,None,None,None,None,2200 CHILDRENS WAY STE 3115,None,NASHVILLE,TN,372320005.0,282NC2000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282NC2000X,General Acute Care Hospital,37232,34980,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,1306889597,2.0,VANDERBILT UNIVERSITY MEDICAL CENTER,None,None,None,None,None,None,1411 W. BADDOUR PARKWAY,None,LEBANON,TN,370872513.0,282N00000X,Y,None,None,None,Non

In [50]:
npis_patient_count.head()

,from_npi,to_npi,patient_count
0,1508085911,1730166125,58
1,1508167040,1730166125,51
2,1508863549,1730166125,340
3,1508867870,1730166125,50
4,1508011040,1730166224,132
